## 数据准备

首先，我们需要准备训练数据，我们有以下选择：

1. 使用sklearn中的手写数字（8*8）；
2. 使用 [tfds](https://github.com/tensorflow/datasets/tree/master) 中的手写数字(28*28) [mnist](https://www.tensorflow.org/datasets/catalog/mnist)。

sklean 中的数字的分辨率比较差，人工识别可能都有困难；我们使用 tfds 中的 mnist 数据集。

In [ ]:
from sklearn import datasets
from matplotlib import pyplot

# sklearn中的手写数据集
digits = datasets.load_digits()

pyplot.gray()
pyplot.matshow(digits.images[8])
# pyplot.matshow(digits.images[16])
pyplot.show()

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import pyplot

# tensorflow中的手写数据集
mnist_ds = tfds.load("mnist", split="train", shuffle_files=True)
a = tfds.as_numpy(mnist_ds.take(10))

pyplot.gray()
pyplot.matshow(a)
# pyplot.matshow(digits.images[16])
pyplot.show()